* Oleksandra Aliyeva
* April 2022
* Find sushi in Boston

In [1]:
#import libraries
import pandas as pd
import numpy as np

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
#load API credentials
with open('/Users/oleksandraaliyeva/.secret/yelp_api.json') as f: 
    login = json.load(f)
#instantiate YelpAPI variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
#set API call parameters and filename before the first call
LOCATION = 'Boston, MA, 02101'
TERM = 'Sushi'

In [5]:
#specifying JSON_FILE filename
JSON_FILE = f'Yelp/results_in_progress_Sushi.json'
JSON_FILE 

'Yelp/results_in_progress_Sushi.json'

In [6]:
#check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
#if it does not exist: 
if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
    #if JSON_FILE included a folder:
    if len(folder)>0:
    #create the folder
        os.makedirs(folder, exist_ok = True)
    
     #INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
     #save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
#if it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Yelp/results_in_progress_Sushi.json not found. Saving empty list to file.


In [7]:
#load previous results and use len of results for offset
with open (JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
#set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
#use yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                               term=TERM,
                               offset=n_results+1)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
#how many results total?
total_results = results['total']
total_results

640

In [10]:
#how many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
#use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/results_per_page)
n_pages

32

In [12]:
#join new results with old list with extend and save to file
previous_results.extend(results['businesses'])
with open(JSON_FILE,'w') as f:
    json.dump(previous_results,f)

In [13]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    #read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    #save number of results for to use as offset
    n_results = len(previous_results)
    #use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results+1)
    
    #append new results and save to file
    previous_results.extend(results['businesses'])
    
    #display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/32 [00:00<?, ?it/s]

In [14]:
#load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(3), final_df.tail(3))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,E3Qqa99qFy1oeGMv0wPBBQ,tora-japanese-restaurant-boston,Tora Japanese Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/ZdaYpg...,False,https://www.yelp.com/biz/tora-japanese-restaur...,553,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 42.35076, 'longitude': -71.06054}","[pickup, delivery]",$$,"{'address1': '20B Tyler St', 'address2': None,...",+16175426688,(617) 542-6688,934.549817
1,Z6Io2AbJrof7TPVliMZkAg,fuji-at-ink-block-boston,Fuji at Ink Block,https://s3-media2.fl.yelpcdn.com/bphoto/xhHJpc...,False,https://www.yelp.com/biz/fuji-at-ink-block-bos...,543,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 42.345077, 'longitude': -71.063505}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '352 Harrison Ave', 'address2': '...",+16179363282,(617) 936-3282,706.357329
2,baoHJnYFntDUICr10aFj8Q,sakana-cambridge-2,Sakana,https://s3-media1.fl.yelpcdn.com/bphoto/wb4xhJ...,False,https://www.yelp.com/biz/sakana-cambridge-2?ad...,242,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 42.3693, 'longitude': -71.11082}","[pickup, delivery]",$$,"{'address1': '983 Massachusetts Ave', 'address...",+16177144646,(617) 714-4646,4030.346045
3,87f7kR7nTz8WHnmtLM_S6w,o-ya-boston,O Ya,https://s3-media4.fl.yelpcdn.com/bphoto/XdZZEb...,False,https://www.yelp.com/biz/o-ya-boston?adjust_cr...,756,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.5,"{'latitude': 42.3514083135866, 'longitude': -7...",[delivery],$$$$,"{'address1': '9 East St Pl', 'address2': '', '...",+16176549900,(617) 654-9900,1246.235064
4,n0NNUVwW-wEKc433oKZX5A,kura-revolving-sushi-bar-watertown,Kura Revolving Sushi Bar,https://s3-media3.fl.yelpcdn.com/bphoto/wluC7r...,False,https://www.yelp.com/biz/kura-revolving-sushi-...,71,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 42.3604231, 'longitude': -71.1898...",[],NaN,"{'address1': '101 Bond St', 'address2': '', 'a...",+16176757575,(617) 675-7575,7446.723594


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
634,RWg7d4FkfoVpjTX6FddwRA,whole-foods-market-newton-4,Whole Foods Market,https://s3-media1.fl.yelpcdn.com/bphoto/GW-Zjr...,False,https://www.yelp.com/biz/whole-foods-market-ne...,56,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",2.5,"{'latitude': 42.330017, 'longitude': -71.206397}",[],$$$,"{'address1': '916 Walnut St', 'address2': '', ...",+16179691141,(617) 969-1141,11284.020874
635,wnEKFzVreBVOX2jANrTDsQ,star-market-brighton,Star Market,https://s3-media3.fl.yelpcdn.com/bphoto/I9FPl0...,False,https://www.yelp.com/biz/star-market-brighton?...,77,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.5,"{'latitude': 42.360928, 'longitude': -71.1375412}",[delivery],$$,"{'address1': '370 Western Ave', 'address2': ''...",+16177875266,(617) 787-5266,5638.822604
636,tZV72RPEG2E24uFulcGjVg,star-market-auburndale,Star Market,https://s3-media4.fl.yelpcdn.com/bphoto/p4SlaX...,False,https://www.yelp.com/biz/star-market-auburndal...,40,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.0,"{'latitude': 42.3467438, 'longitude': -71.2447...",[delivery],$$,"{'address1': '2040 Commonwealth Ave', 'address...",+16179651793,(617) 965-1793,14258.843346
637,m6ZCINThm8PwkK91qJcrPw,trader-joes-arlington,Trader Joe's,https://s3-media1.fl.yelpcdn.com/bphoto/KwDTvS...,False,https://www.yelp.com/biz/trader-joes-arlington...,96,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.5,"{'latitude': 42.4254178, 'longitude': -71.1888...",[],$$,"{'address1': '1427 Massachusetts Ave', 'addres...",+17816469138,(781) 646-9138,12948.424375
638,MpfauJengAZHTzoIkwP_yw,the-cheesecake-factory-burlington,The Cheesecake Factory,https://s3-media1.fl.yelpcdn.com/bphoto/95h89v...,False,https://www.yelp.com/biz/the-cheesecake-factor...,529,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",3.0,"{'latitude': 42.4822898, 'longitude': -71.2135...","[restaurant_reservation, delivery]",$$,"{'address1': '75 Middlesex Turnpike', 'address...",+17812730060,(781) 273-0060,18972.757453


In [15]:
#check for duplicate IDs
final_df.duplicated(subset='id').sum()

1

In [16]:
final_df.drop_duplicates(subset='id', keep='first', inplace = True)

In [17]:
final_df.duplicated(subset='id').sum()

0

In [18]:
# save the final results to a compressed csv
final_df.to_csv('Yelp/final_results_sushi.csv.gz', compression='gzip',
                index=False)